# API Service Core Functionality Test

This notebook tests the core functionality of the GroupNegotiationService and related API endpoints.

## Test Coverage:
- Service initialization and database connections
- Group negotiation startup
- Session management and retrieval
- Message sending functionality
- Tenant interest simulation
- Negotiation statistics
- Error handling

In [1]:
# Import required modules
import sys
import os
import asyncio
import json
from datetime import datetime

# Add the app directory to Python path
sys.path.append('/home/yushiran/Rental_Agent/backend')

# Import API service components
from app.api_service.group_negotiation import GroupNegotiationService
from app.mongo.client import MongoClientWrapper

print("✅ Successfully imported API service modules")
print(f"⏰ Test started at: {datetime.now()}")

2025-06-06 18:05:43.566 | INFO     | app.rag.retrievers:get_retriever:30 - Initializing retriever | model: sentence-transformers/all-MiniLM-L6-v2 | device: cpu | top_k: 3


✅ Successfully imported API service modules
⏰ Test started at: 2025-06-06 18:05:45.740813


In [2]:
# Initialize services and test database connections
try:
    # Initialize GroupNegotiationService
    group_service = GroupNegotiationService()
    print("✅ GroupNegotiationService initialized successfully")
    
    # Test database connections through GroupNegotiationService
    print("\n🔗 Testing database connections...")
    
    # Test tenants collection
    tenants_count = len(group_service.tenants_db.fetch_documents(limit=1,query={}))
    print(f"📊 Tenants collection accessible: {tenants_count >= 0}")
    
    # Test landlords collection
    landlords_count = len(group_service.landlords_db.fetch_documents(limit=1,query={}))
    print(f"📊 Landlords collection accessible: {landlords_count >= 0}")
    
    # Test properties collection
    properties_count = len(group_service.properties_db.fetch_documents(limit=1,query={}))
    print(f"📊 Properties collection accessible: {properties_count >= 0}")
    
    initialization_success = True
    print("\n✅ All services and database connections initialized successfully")
    
except Exception as e:
    initialization_success = False
    print(f"❌ Service initialization failed: {e}")

2025-06-06 18:05:45.763 | INFO     | app.mongo.client:__init__:86 - Connected to MongoDB instance:
 URI: mongodb://localhost:27017/rental_agent
 Database: rental_agent
 Collection: landlords
2025-06-06 18:05:45.769 | INFO     | app.mongo.client:__init__:86 - Connected to MongoDB instance:
 URI: mongodb://localhost:27017/rental_agent
 Database: rental_agent
 Collection: tenants
2025-06-06 18:05:45.774 | INFO     | app.mongo.client:__init__:86 - Connected to MongoDB instance:
 URI: mongodb://localhost:27017/rental_agent
 Database: rental_agent
 Collection: properties
2025-06-06 18:05:45.809 | INFO     | app.mongo.client:__init__:86 - Connected to MongoDB instance:
 URI: mongodb://localhost:27017/rental_agent
 Database: rental_agent
 Collection: landlords
2025-06-06 18:05:45.815 | INFO     | app.mongo.client:__init__:86 - Connected to MongoDB instance:
 URI: mongodb://localhost:27017/rental_agent
 Database: rental_agent
 Collection: tenants
2025-06-06 18:05:45.821 | INFO     | app.mongo.c

✅ GroupNegotiationService initialized successfully

🔗 Testing database connections...
📊 Tenants collection accessible: True
📊 Landlords collection accessible: True
📊 Properties collection accessible: True

✅ All services and database connections initialized successfully


In [3]:
# Check data availability for testing
if initialization_success:
    try:
        print("📋 Checking data availability for testing...\n")
        
        # Get available tenants
        tenants = group_service.tenants_db.fetch_documents(limit=5, query={})
        print(f"👥 Available tenants: {len(tenants)}")
        if tenants:
            # Access attributes directly since these are now Pydantic models
            sample_tenant = tenants[0]
            print(f"   Sample tenant: {sample_tenant.name} (ID: {sample_tenant.tenant_id})")
        
        # Get available landlords
        landlords = group_service.landlords_db.fetch_documents(limit=5, query={})
        print(f"🏠 Available landlords: {len(landlords)}")
        if landlords:
            # Access attributes directly since these are now Pydantic models
            sample_landlord = landlords[0]
            print(f"   Sample landlord: {sample_landlord.name} (ID: {sample_landlord.landlord_id})")
        
        # Get available properties
        properties = group_service.properties_db.fetch_documents(limit=5, query={})
        print(f"🏘️ Available properties: {len(properties)}")
        if properties:
            # Access attributes directly since these are now Pydantic models
            sample_property = properties[0]
            print(f"   Sample property: {sample_property.display_address} (ID: {sample_property.property_id})")
        
        # Check active sessions in memory
        active_sessions = await group_service.get_all_active_sessions()
        print(f"💬 Active sessions in memory: {len(active_sessions)}")
        
        data_available = len(tenants) > 0 and len(landlords) > 0 and len(properties) > 0
        print(f"\n📊 Data sufficient for testing: {data_available}")
        
    except Exception as e:
        data_available = False
        print(f"❌ Data availability check failed: {e}")
        import traceback
        traceback.print_exc()
else:
    data_available = False
    print("⏭️ Skipping data availability check due to initialization failure")

2025-06-06 18:05:45.989 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 5 documents with query: {}
2025-06-06 18:05:45.991 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 5 documents with query: {}
2025-06-06 18:05:45.993 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 5 documents with query: {}


📋 Checking data availability for testing...

👥 Available tenants: 5
   Sample tenant: Emily Jones (ID: 72f3d510-09d0-4763-aa07-9789fe1abb9c)
🏠 Available landlords: 5
   Sample landlord: Home Made (ID: 99a11d9e-451b-4d92-a0b3-7a8033762b33)
🏘️ Available properties: 5
   Sample property: Makers House, Stratford, E20 (ID: 162453242)
💬 Active sessions in memory: 0

📊 Data sufficient for testing: True


In [4]:
# Test group negotiation startup functionality
if initialization_success and data_available:
    try:
        print("🚀 Testing group negotiation startup...\n")
        
        # Start group negotiation with a small number for testing
        max_tenants = 3
        print(f"Starting group negotiation for maximum {max_tenants} tenants...")
        
        # Use asyncio to run the async function
        result = await group_service.start_group_negotiation(max_tenants=max_tenants)
        
        print("📊 Group negotiation startup results:")
        print(f"   Total tenants: {result.get('total_tenants', 0)}")
        print(f"   Total landlords: {result.get('total_landlords', 0)}")
        print(f"   Successful matches: {result.get('successful_matches', 0)}")
        
        if 'sessions' in result:
            print(f"   Created sessions: {len(result['sessions'])}")
            for i, session in enumerate(result['sessions'][:2]):
                print(f"   Session {i+1}: {session.get('session_id', 'Unknown')}")
        
        group_negotiation_success = 'error' not in result
        if group_negotiation_success:
            print("\n✅ Group negotiation startup successful")
        else:
            print(f"\n⚠️ Group negotiation startup had issues: {result.get('error')}")
            
    except Exception as e:
        group_negotiation_success = False
        print(f"❌ Group negotiation startup failed: {e}")
else:
    group_negotiation_success = False
    print("⏭️ Skipping group negotiation startup test due to previous failures")

2025-06-06 18:05:46.000 | INFO     | app.api_service.group_negotiation:start_group_negotiation:52 - 开始群体协商流程...
2025-06-06 18:05:46.002 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 3 documents with query: {}
2025-06-06 18:05:46.010 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 49 documents with query: {}
2025-06-06 18:05:46.015 | INFO     | app.api_service.group_negotiation:_match_tenants_to_landlords:127 - 为 1 个租客找到了匹配的房东
2025-06-06 18:05:46.016 | INFO     | app.api_service.group_negotiation:_start_negotiation_session:186 - 创建协商会话 90d1e7d9-86b3-4704-ad73-288c4378b645: Joyce Reeves <-> Home Made (房产: Farnsworth Court, West Parkside, LONDON, SE10)
2025-06-06 18:05:46.016 | INFO     | app.api_service.group_negotiation:start_group_negotiation:74 - 成功启动 1 个协商会话


🚀 Testing group negotiation startup...

Starting group negotiation for maximum 3 tenants...
📊 Group negotiation startup results:
   Total tenants: 3
   Total landlords: 49
   Successful matches: 1
   Created sessions: 1
   Session 1: 90d1e7d9-86b3-4704-ad73-288c4378b645

✅ Group negotiation startup successful


In [5]:
# Test session management functionality
if initialization_success:
    try:
        print("📋 Testing session management functionality...\n")
        
        # Get all active sessions
        print("Retrieving all active sessions...")
        active_sessions = await group_service.get_all_active_sessions()
        
        print(f"📊 Found {len(active_sessions)} active session(s)")
        
        if active_sessions:
            # Test getting specific session info
            # Handle both dict and object responses for session
            first_session = active_sessions[0]
            if hasattr(first_session, 'session_id'):
                test_session_id = first_session.session_id
            else:
                test_session_id = first_session.get('session_id')
            
            print(f"\n🔍 Testing session info retrieval for: {test_session_id}")
            
            session_info = await group_service.get_session_info(test_session_id)
            
            if session_info:
                print("✅ Session info retrieved successfully:")
                # Handle both dict and object responses
                if hasattr(session_info, 'tenant_name'):
                    print(f"   Tenant: {getattr(session_info, 'tenant_name', 'Unknown')}")
                    print(f"   Landlord: {getattr(session_info, 'landlord_name', 'Unknown')}")
                    print(f"   Property: {getattr(session_info, 'property_address', 'Unknown')}")
                    print(f"   Rent: ${getattr(session_info, 'monthly_rent', 0)}")
                    print(f"   Status: {getattr(session_info, 'status', 'Unknown')}")
                else:
                    print(f"   Tenant: {session_info.get('tenant_name', 'Unknown')}")
                    print(f"   Landlord: {session_info.get('landlord_name', 'Unknown')}")
                    print(f"   Property: {session_info.get('property_address', 'Unknown')}")
                    print(f"   Rent: ${session_info.get('monthly_rent', 0)}")
                    print(f"   Status: {session_info.get('status', 'Unknown')}")
                session_management_success = True
            else:
                print("⚠️ Session info retrieval returned empty result")
                session_management_success = False
        else:
            print("ℹ️ No active sessions found to test session info retrieval")
            session_management_success = True  # Not a failure, just no data
            
    except Exception as e:
        session_management_success = False
        print(f"❌ Session management test failed: {e}")
        import traceback
        traceback.print_exc()
else:
    session_management_success = False
    print("⏭️ Skipping session management test due to initialization failure")

📋 Testing session management functionality...

Retrieving all active sessions...
📊 Found 1 active session(s)

🔍 Testing session info retrieval for: 90d1e7d9-86b3-4704-ad73-288c4378b645
✅ Session info retrieved successfully:
   Tenant: Joyce Reeves
   Landlord: Home Made
   Property: Farnsworth Court, West Parkside, LONDON, SE10
   Rent: $2050
   Status: active


In [6]:
# Test message sending functionality
if initialization_success and session_management_success:
    try:
        print("💬 Testing message sending functionality...\n")
        
        # Get an active session for testing
        active_sessions = await group_service.get_all_active_sessions()
        
        if active_sessions:
            test_session = active_sessions[0]
            # Handle both dict and object responses
            if hasattr(test_session, 'session_id'):
                session_id = test_session.session_id
                tenant_id = test_session.tenant_id
                tenant_name = getattr(test_session, 'tenant_name', 'Unknown')
            else:
                session_id = test_session.get('session_id')
                tenant_id = test_session.get('tenant_id')
                tenant_name = test_session.get('tenant_name', 'Unknown')
            
            print(f"📝 Sending test message to session: {session_id}")
            print(f"   From tenant: {tenant_name}")
            
            # Send a test message
            test_message = "Hello! I'm interested in this property. Could you tell me more about the neighborhood?"
            
            result = await group_service.send_message_to_session(
                session_id=session_id,
                sender_id=tenant_id,
                message=test_message,
                sender_type="tenant"
            )
            
            if 'error' not in result:
                print("✅ Message sent successfully:")
                print(f"   Response received: {len(result.get('response', ''))} characters")
                print(f"   Message count: {result.get('message_count', 0)}")
                print(f"   Response chunks: {len(result.get('response_chunks', []))}")
                
                # Show first 100 characters of response
                response_preview = result.get('response', '')[:100]
                if response_preview:
                    print(f"   Response preview: {response_preview}...")
                    
                message_sending_success = True
            else:
                print(f"⚠️ Message sending had error: {result['error']}")
                message_sending_success = False
                
        else:
            print("ℹ️ No active sessions available for message testing")
            message_sending_success = True  # Not a failure, just no data
            
    except Exception as e:
        message_sending_success = False
        print(f"❌ Message sending test failed: {e}")
        import traceback
        traceback.print_exc()
else:
    message_sending_success = False
    print("⏭️ Skipping message sending test due to previous failures")

2025-06-06 18:05:46.037 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 1 documents with query: {'landlord_id': '4df199e9-b05e-4eec-9a83-1fcfb31e495f'}


💬 Testing message sending functionality...

📝 Sending test message to session: 90d1e7d9-86b3-4704-ad73-288c4378b645
   From tenant: Joyce Reeves


OPIK: You didn't specify argument `stream_usage`=True during LLM initialization. Token usage data is not available for .stream() or .astream() methods.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974634-aa48-7ac2-8a7b-2b99395cd8c9&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


✅ Message sent successfully:
   Response received: 297 characters
   Message count: 2
   Response chunks: 53
   Response preview: To provide you with accurate information about the neighborhood, I need to know which property you a...


In [7]:
# Test tenant interest simulation
if initialization_success:
    try:
        print("🎭 Testing tenant interest simulation...\n")
        
        # Simulate tenant interests
        print("Simulating tenant interests for all active sessions...")
        
        result = await group_service.simulate_all_tenant_interests()
        
        print("📊 Tenant interest simulation results:")
        print(f"   Total simulations run: {len(result)}")
        
        successful_simulations = 0
        failed_simulations = 0
        
        for i, sim_result in enumerate(result[:3]):  # Show up to 3 results
            if 'error' not in sim_result.get('result', {}):
                successful_simulations += 1
                status = "✅"
            else:
                failed_simulations += 1
                status = "❌"
                
            print(f"   {status} Session {i+1}: {sim_result.get('session_id', 'Unknown')}")
            if 'error' in sim_result.get('result', {}):
                print(f"      Error: {sim_result['result']['error']}")
        
        print(f"\n📊 Summary:")
        print(f"   Successful simulations: {successful_simulations}")
        print(f"   Failed simulations: {failed_simulations}")
        
        simulation_success = failed_simulations == 0 or successful_simulations > 0
        if simulation_success:
            print("\n✅ Tenant interest simulation completed successfully")
        else:
            print("\n⚠️ Tenant interest simulation had issues")
            
    except Exception as e:
        simulation_success = False
        print(f"❌ Tenant interest simulation failed: {e}")
else:
    simulation_success = False
    print("⏭️ Skipping tenant interest simulation due to initialization failure")

2025-06-06 18:05:47.313 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 1 documents with query: {'landlord_id': '4df199e9-b05e-4eec-9a83-1fcfb31e495f'}


🎭 Testing tenant interest simulation...

Simulating tenant interests for all active sessions...
📊 Tenant interest simulation results:
   Total simulations run: 1
   ✅ Session 1: 90d1e7d9-86b3-4704-ad73-288c4378b645

📊 Summary:
   Successful simulations: 1
   Failed simulations: 0

✅ Tenant interest simulation completed successfully


In [8]:
# Test negotiation statistics functionality
if initialization_success:
    try:
        print("📊 Testing negotiation statistics retrieval...\n")
        
        # Get negotiation statistics
        stats = group_service.get_negotiation_stats()
        
        print("📈 Negotiation Statistics:")
        print(f"   Active sessions: {stats.get('active_sessions', 0)}")
        print(f"   Total messages: {stats.get('total_messages', 0)}")
        print(f"   Average messages per session: {stats.get('average_messages_per_session', 0)}")
        print(f"   Average match score: {stats.get('average_match_score', 0)}")
        
        statistics_success = True
        print("\n✅ Negotiation statistics retrieved successfully")
        
    except Exception as e:
        statistics_success = False
        print(f"❌ Negotiation statistics test failed: {e}")
else:
    statistics_success = False
    print("⏭️ Skipping negotiation statistics test due to initialization failure")

📊 Testing negotiation statistics retrieval...

📈 Negotiation Statistics:
   Active sessions: 1
   Total messages: 4
   Average messages per session: 4.0
   Average match score: 90.0

✅ Negotiation statistics retrieved successfully


In [9]:
# Test error handling
if initialization_success:
    try:
        print("🛡️ Testing error handling...\n")
        
        error_handling_results = []
        
        # Test 1: Invalid session ID
        print("Test 1: Getting info for non-existent session...")
        try:
            invalid_session = await group_service.get_session_info("invalid-session-id")
            test1_result = invalid_session is None
            print(f"   ✅ Properly handled invalid session ID: {test1_result}")
        except Exception as e:
            test1_result = True  # Exception is acceptable for invalid input
            print(f"   ✅ Exception raised for invalid session ID: {type(e).__name__}")
        error_handling_results.append(test1_result)
        
        # Test 2: Message to non-existent session
        print("\nTest 2: Sending message to non-existent session...")
        try:
            result = await group_service.send_message_to_session(
                session_id="invalid-session-id",
                sender_id="invalid-sender",
                message="Test message",
                sender_type="tenant"
            )
            test2_result = 'error' in result
            print(f"   ✅ Error properly returned: {test2_result}")
            if 'error' in result:
                print(f"   Error message: {result['error']}")
        except Exception as e:
            test2_result = True  # Exception is acceptable
            print(f"   ✅ Exception raised for invalid message: {type(e).__name__}")
        error_handling_results.append(test2_result)
        
        # Test 3: Group negotiation with invalid parameters
        print("\nTest 3: Group negotiation with invalid parameters...")
        try:
            result = await group_service.start_group_negotiation(max_tenants=0)
            test3_result = 'error' in result or result.get('total_processed', 0) == 0
            print(f"   ✅ Handled invalid parameters: {test3_result}")
        except Exception as e:
            test3_result = True  # Exception is acceptable
            print(f"   ✅ Exception raised for invalid parameters: {type(e).__name__}")
        error_handling_results.append(test3_result)
        
        error_handling_success = all(error_handling_results)
        print(f"\n📊 Error handling tests: {sum(error_handling_results)}/{len(error_handling_results)} passed")
        
        if error_handling_success:
            print("✅ Error handling tests completed successfully")
        else:
            print("⚠️ Some error handling tests failed")
            
    except Exception as e:
        error_handling_success = False
        print(f"❌ Error handling tests failed: {e}")
else:
    error_handling_success = False
    print("⏭️ Skipping error handling tests due to initialization failure")

2025-06-06 18:05:49.987 | INFO     | app.api_service.group_negotiation:start_group_negotiation:52 - 开始群体协商流程...
2025-06-06 18:05:49.989 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 50 documents with query: {}


2025-06-06 18:05:49.995 | DEBUG    | app.mongo.client:fetch_documents:173 - Fetched 49 documents with query: {}
2025-06-06 18:05:50.045 | INFO     | app.api_service.group_negotiation:_match_tenants_to_landlords:127 - 为 32 个租客找到了匹配的房东
2025-06-06 18:05:50.046 | INFO     | app.api_service.group_negotiation:_start_negotiation_session:186 - 创建协商会话 1031f046-cd6f-427c-a180-042849d13b6b: Joyce Reeves <-> Home Made (房产: Farnsworth Court, West Parkside, LONDON, SE10)
2025-06-06 18:05:50.046 | INFO     | app.api_service.group_negotiation:_start_negotiation_session:186 - 创建协商会话 9f3fe888-2289-46f7-988e-4bc8b6cba319: Dr Gillian Chan <-> Hanley Estates Ltd (房产: Timbrell Place, London, SE16)
2025-06-06 18:05:50.047 | INFO     | app.api_service.group_negotiation:_start_negotiation_session:186 - 创建协商会话 d93c843e-1f15-4b05-9d30-5fe764bf8792: Richard Foster-Thompson <-> Ariston Property (房产: Comeragh Road, London)
2025-06-06 18:05:50.047 | INFO     | app.api_service.group_negotiation:_start_negotiation_ses

🛡️ Testing error handling...

Test 1: Getting info for non-existent session...
   ✅ Properly handled invalid session ID: True

Test 2: Sending message to non-existent session...
   ✅ Error properly returned: True
   Error message: 协商会话不存在

Test 3: Group negotiation with invalid parameters...
   ✅ Handled invalid parameters: True

📊 Error handling tests: 3/3 passed
✅ Error handling tests completed successfully


In [10]:
# Test Summary
print("\n" + "="*60)
print("📋 API SERVICE CORE FUNCTIONALITY TEST SUMMARY")
print("="*60)

# Collect all test results
test_results = {
    "Service Initialization": initialization_success,
    "Data Availability": data_available if 'data_available' in locals() else False,
    "Group Negotiation Startup": group_negotiation_success if 'group_negotiation_success' in locals() else False,
    "Session Management": session_management_success if 'session_management_success' in locals() else False,
    "Message Sending": message_sending_success if 'message_sending_success' in locals() else False,
    "Tenant Interest Simulation": simulation_success if 'simulation_success' in locals() else False,
    "Negotiation Statistics": statistics_success if 'statistics_success' in locals() else False,
    "Error Handling": error_handling_success if 'error_handling_success' in locals() else False
}

# Display results
passed_tests = 0
total_tests = len(test_results)

for test_name, result in test_results.items():
    status = "✅ PASS" if result else "❌ FAIL"
    print(f"{status} {test_name}")
    if result:
        passed_tests += 1

print("\n" + "-"*60)
print(f"📊 OVERALL RESULTS: {passed_tests}/{total_tests} tests passed")

success_rate = (passed_tests / total_tests) * 100 if total_tests > 0 else 0
print(f"📈 Success Rate: {success_rate:.1f}%")

if success_rate >= 80:
    print("🎉 API Service core functionality is working well!")
elif success_rate >= 60:
    print("⚠️ API Service has some issues that need attention")
else:
    print("🚨 API Service has significant issues that require investigation")

print(f"\n⏰ Test completed at: {datetime.now()}")
print("="*60)


📋 API SERVICE CORE FUNCTIONALITY TEST SUMMARY
✅ PASS Service Initialization
✅ PASS Data Availability
✅ PASS Group Negotiation Startup
✅ PASS Session Management
✅ PASS Message Sending
✅ PASS Tenant Interest Simulation
✅ PASS Negotiation Statistics
✅ PASS Error Handling

------------------------------------------------------------
📊 OVERALL RESULTS: 8/8 tests passed
📈 Success Rate: 100.0%
🎉 API Service core functionality is working well!

⏰ Test completed at: 2025-06-06 18:05:50.065690
